# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weatherpy_data = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Drop unnamed column (unnecessary)

weatherpy_df = weatherpy_data.drop(columns=["Unnamed: 0"])

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weatherpy_df[['Latitude','Longtitude']]
humidity = weatherpy_df['Humidity'].astype(float)

fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations, weights = humidity,
                             dissipating = False, max_intensity = 10,
                             point_radius = 1)

# Add heatmap layer to the map
fig.add_layer(heatmap)
# display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
ideal_cities = pd.DataFrame(weatherpy_data, columns = ["City", "Country", "Latitude", "Longtitude", "Max Temp", "Humidity", "Cloudiness"])

max_temp = (ideal_cities["Max Temp"] <= 90) & (ideal_cities["Max Temp"] >= 50)
humidity = (ideal_cities["Humidity"] < 50)
cloudiness = (ideal_cities["Cloudiness"] <= 50)
lat_ideal = (ideal_cities["Latitude"])
long_ideal= (ideal_cities["Longtitude"])

ideal_cities_df= ideal_cities[max_temp & humidity & cloudiness]
ideal_cities_df

,City,Country,Latitude,Longtitude,Max Temp,Humidity,Cloudiness
19,Bonanza,NI,14.0289,-84.5910,87.51,37,7
85,Adrar,MR,20.5022,-10.0711,79.07,12,0
89,Solapur,IN,17.6833,75.9167,74.68,28,13
133,Malumfashi,NG,11.7853,7.6218,75.31,17,2
141,Ukwā,IN,21.9667,80.4667,63.16,31,0
145,Mayahi,NE,13.9553,7.6712,80.02,14,0
174,Kidal,ML,18.4411,1.4078,81.70,13,0
190,Cocorit,MX,27.5833,-109.9667,69.80,46,20
205,Hīt,IQ,33.6416,42.8251,64.00,39,0
219,Shwebo,MM,22.5667,95.7000,69.15,42,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = pd.DataFrame(ideal_cities_df, columns =["City", "Country", "Latitude", "Longtitude"])

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longtitude,Hotel Name
19,Bonanza,NI,14.0289,-84.5910,
85,Adrar,MR,20.5022,-10.0711,
89,Solapur,IN,17.6833,75.9167,
133,Malumfashi,NG,11.7853,7.6218,
141,Ukwā,IN,21.9667,80.4667,


In [16]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longtitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 19: Bonanza.
Closest hotel is Edificio Administrativo HEMCO.
------------
Retrieving Results for Index 85: Adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 89: Solapur.
Closest hotel is SPOT ON 43891 Vinayak Lodge SPOT.
------------
Retrieving Results for Index 133: Malumfashi.
Closest hotel is INFINITY CYBER CAFE MALUMFASHI.
------------
Retrieving Results for Index 141: Ukwā.
Closest hotel is Chouksey Hotel & Rajat Transport.
------------
Retrieving Results for Index 145: Mayahi.
Missing field/result... skipping.
------------
Retrieving Results for Index 174: Kidal.
Missing field/result... skipping.
------------
Retrieving Results for Index 190: Cocorit.
Closest hotel is Kaba Motel.
------------
Retrieving Results for Index 205: Hīt.
Closest hotel is صالون زيرو.
------------
Retrieving Results for Index 219: Shwebo.
Closest hotel is MOON SHINE Guest House.
------------
Retrieving Results for Index 236: Lázaro Cárdenas.
Cl

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

TypeError: list indices must be integers or slices, not list

In [ ]:
# Add marker layer ontop of heat map


# Display figure
